In [5]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import sys
import yaml

SCRIPT_DIR = os.getcwd()
sys.path.append(os.path.dirname(SCRIPT_DIR))
print(SCRIPT_DIR) 

from visualisation_module import *
from observation_module import *
from misc.helpers import *

PATH_TO_PLOTS = '../plots/'
path_to_results = '../results/'

#EX_CONFIG = load_config(config_path = '../../configs/smosmania_st.yaml')
EX_CONFIG = load_config(config_path = '../../configs/smosmania_st.yaml')
EX_STATION = "Condom"

# List of paths to the .nc files
# '/perm/dadf/HSAF_validation/in_situ_data/pre_processed_data/ismn_nc/soil_TERENO_ISMN_2022.nc'
#dir_to_data = "/perm/dadf/HSAF_validation/in_situ_data/pre_processed_data/ismn_nc"
station_name = 'soil_SMOSMANIA_ISMN_2008.nc'

all_stations = ['CabrieresdAvignon', 'Condom', 'CreondArmagnac', 'LaGrandCombe',
       'Lahas', 'LezignanCorbieres', 'Mazan-Abbaye', 'Mejannes-le-Clap',
       'Montaut', 'Mouthoumet', 'Narbonne', 'PeyrusseGrande', 'Pezenas-old',
       'Sabres', 'SaintFelixdeLauragais', 'Savenes', 'Urgons', 'Villevielle']
use_stations = ['Condom', 'Villevielle', 'LaGrandCombe', 'Narbonne']

Station = ObservationModule(network = EX_CONFIG['network'], 
                                station = EX_STATION ,
                                variable = EX_CONFIG['variable'],
                                depth=EX_CONFIG['depth']) # Initialise the Observation Module with the default Station (Gevenich)
    

Station.load_station(years = EX_CONFIG['years']) # Load two years of station data for lookback slicing
Station.load_forcing() # Load forcing for matching data base with station data
closest_grid_cell = Station.match_station_with_forcing() # Match the station with clostest grid cell and extract the index of the grid cell
Station.process_station_data() # specify path_to_plots, if you want to visualise
station_data = Station.slice_station_data(lookback=0,t_0=EX_CONFIG['initial_time'])

DOY_VECTOR = Station.doy_vector

/etc/ecmwf/nfs/dh1_home_a/pamw/PycharmProjects/forecast-horizons/ecland-emulator/notebooks
Opening ../../configs/smosmania_st.yaml for experiment configuration.
Select station:  Condom
Select depth:  [0.05, 0.2, 0.3]
Extracted year: 2022
Matched station with grid cell:  6901
Converting celsius into kelvin
Resampling to 6-hourly mean.
Selecting from index: 1584
Subtract lookback for new index: 1584


In [9]:
stations_dict = {}
for station in use_stations:
    with open(f"../results/SMOSMANIA_{station}_2022_st_ensemble.yaml", 'r') as f:
        layers = yaml.load(f, Loader=yaml.UnsafeLoader)
    stations_dict[station] = layers

forecast_dict = {}
for station in use_stations[:7]:
    with open(f"../results/SMOSMANIA_{station}_2022_st_ensemble_fc.yaml", 'r') as f:
        layers_fc = yaml.load(f, Loader=yaml.UnsafeLoader)
    forecast_dict[station] = layers_fc

In [4]:
print(stations_dict)

{'Condom': {'layer0': {'scores': {'ECLand': array([0.4987793 , 2.380127  , 0.11517334, 0.05633545, 0.29159546,
       0.36834717, 1.1132812 , 0.8353882 , 0.7505493 , 2.6532288 ,
       0.9769287 , 1.2708435 , 0.5637207 , 1.7784424 , 0.60946655,
       0.02279663, 0.67071533, 0.7395935 , 0.06359863, 0.490448  ,
       1.7929993 , 2.4849243 , 0.49145508, 0.0899353 , 0.9779663 ,
       0.50598145, 0.76626587, 0.1461792 , 2.7624512 , 4.286194  ,
       0.50439453, 0.4885559 , 2.5143433 , 2.831604  , 0.3451538 ,
       0.6279602 , 1.8383484 , 3.417389  , 1.0570984 , 0.6584778 ,
       1.6108093 , 0.97314453, 0.4906006 , 0.32391357, 2.9355469 ,
       4.2069397 , 1.2388611 , 0.49777222, 1.520752  , 1.655426  ,
       0.49334717, 0.97320557, 0.7249756 , 1.3523865 , 0.10098267,
       0.35772705], dtype=float32), 'Emulators': array([0.19105021, 1.53349678, 0.58095535, 0.26246997, 0.20502534,
       0.31095817, 0.53281456, 1.34559947, 1.01837073, 2.28395866,
       0.78105769, 1.71210322, 0.470

In [18]:
PlotStations = VisualisationMany(
                 network = EX_CONFIG["network"], 
                 station = "all", 
                 variable = EX_CONFIG["variable"], 
                 maximum_leadtime = 672, 
                 score = EX_CONFIG["score"],
                 doy_vector = DOY_VECTOR,
                 evaluation = "ens", 
                 path_to_plots = PATH_TO_PLOTS
)

PlotStations.assemble_scores(stations_dict)


print(np.argmax((1 - PlotStations.scores_l1["skill_scores_upper"]) < 0))
print(np.argmax((1 - PlotStations.scores_l1["skill_scores_mean"]) < 0))
print(np.argmax((1 - PlotStations.scores_l1["skill_scores_lower"]) < 0))

print(np.argmax((1 - PlotStations.scores_l2["skill_scores_upper"]) < 0))
print(np.argmax((1 - PlotStations.scores_l2["skill_scores_mean"]) < 0))
print(np.argmax((1 - PlotStations.scores_l2["skill_scores_lower"]) < 0))

print(np.argmax((1 - PlotStations.scores_l3["skill_scores_upper"]) < 0))
print(np.argmax((1 - PlotStations.scores_l3["skill_scores_mean"]) < 0))
print(np.argmax((1 - PlotStations.scores_l3["skill_scores_lower"]) < 0))


1
4
8
1
10
57
0
15
300
